# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set

## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [262]:
import re
from collections import defaultdict, Counter

## Classical Norvig's solution

### Loading the Text Corpus
To build a language model, we need a large text corpus. 

The script downloads a large text file from `norvig.com`, processes it, and constructs a dictionary of words and their frequencies.


In [263]:
import requests

url = "https://norvig.com/big.txt"
r = requests.get(url)
r.raise_for_status()

In [264]:
def read_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

big_text = read_corpus('big.txt')

### Processing the Corpus
The text is converted into lowercase and tokenized into words using regular expressions. 

A `Counter` object is used to store word frequencies, which helps in determining word probabilities.


### Building a Probability Model for Words
The probability of a word appearing in the corpus is calculated as:

$P(w) = \frac{\text{count of } w}{\text{total words in corpus}}$

This helps in ranking candidate words based on their likelihood in real-world usage.


In [265]:
def words(text):
    return re.findall(r'[a-z]+', text.lower())

WORDS = Counter(words(big_text)) 
N = sum(WORDS.values())

def P(word):
    return WORDS[word] / N

### Generating Possible Word Corrections
A series of functions are implemented to generate possible corrections for a given word:
- **Edits1:** Generates words that are one edit away (deletion, transposition, replacement, insertion).
- **Edits2:** Generates words that are two edits away.
- **Known Words Filter:** Filters only valid words found in the corpus.
- **Candidates Selection:** Chooses the best correction based on probability.


### Implementing the Spelling Correction Function
The `correct_word` function takes a misspelled word and returns the most probable correction. It works by:
1. Checking if the word itself is known.
2. Checking for words that are one edit away.
3. Checking for words that are two edits away.
4. Returning the most probable word.


In [266]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words_list):
    return set(w for w in words_list if w in WORDS)

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or {word})

def correct_word(word):
    return max(candidates(word), key=P) if candidates(word) else word

### Loading the Evaluation Dataset
The test dataset is loaded to evaluate the accuracy of the spell checker. The dataset consists of:
- A column with correct text (`text`).
- A column with augmented text containing spelling errors (`augmented_text`).


In [267]:
import pandas as pd

evaluation_data = pd.read_csv('test.csv')

evaluation_data.head()

,text,augmented_text
0,project looks to mulesing genetic alternative,project looks to muelsnig ngeetic alternative
1,chemical agents used during protest at port au...,chemical agents used during LrotWst at port ah...
2,business chamber seeks budget infrastructure b...,business hcmaber seeks budget infrastrcutuer b...
3,3600 trips made to darwin tip after cyclone,3600 trips made to adrwni tip after cyconle
4,go between bridge to open july 5,go net3een brisye to lprn july 5


### Defining Evaluation Parameters
The variable `SLICE_SIZE` is set to 10,000, which controls the number of samples from the dataset that will be used for evaluation.


In [268]:
SLICE_SIZE = 10000

### Evaluating the Spell Checker
The `evaluate_baseline` function runs the spell checker on a sample of `SLICE_SIZE` entries from the dataset.

The workflow includes:
- Iterating through each row of test data.
- Applying the `correct_word` function to each token in the augmented text.
- Calculating the Word Error Rate (WER) by comparing the corrected text with the reference text.
- Computing the accuracy as `1 - WER` and averaging it over all test samples.


In [ ]:
import pandas as pd
from jiwer import wer
from tqdm import tqdm


def evaluate_baseline(slice_size):

    evaluation_data = pd.read_csv('data/test.csv')
    data = evaluation_data.head(slice_size)

    norvig_scores = []
    for idx, row in tqdm(data.iterrows(), desc="Norvig baseline evaluation", total=len(data)):
        ref = row["text"]  
        tokens = row["augmented_text"].split()
        
        corrected_tokens = [correct_word(t) for t in tokens]
        
        hyp = " ".join(corrected_tokens)
        
        score = wer(ref, hyp)
        acc = 1 - score
        norvig_scores.append(acc)

    overall_acc_norvig = sum(norvig_scores)/len(norvig_scores)
    print("Average Accuracy with Norvig baseline:", overall_acc_norvig)

    return overall_acc_norvig
evaluate_baseline(slice_size = SLICE_SIZE)

Norvig baseline evaluation: 100%|██████████| 10000/10000 [20:27<00:00,  8.15it/s]

Average Accuracy with Norvig baseline: 0.702366524031524


0.702366524031524

### Norvig's Original Solution: Conclusion

The spell checker based on Norvig’s approach is **reasonably accurate**: `70.2%`, but there is still **room for improvement** in both performance and accuracy.

However, one major drawback is its **speed**—it processes an average of **8 rows per second**, which is relatively slow for large datasets.

### Next Steps: Enhancing the Model

To improve accuracy and efficiency, we will implement the following enhancements:

- **N-gram Models (up to 5-grams):** Introducing context-aware corrections by considering word sequences rather than isolated words.
- **Add-One Smoothing:** Ensuring better probability estimates for unseen n-grams.
- **Beam Search:** Using a probabilistic search strategy to improve sentence-level corrections.

These optimizations will provide more **contextually accurate** and **faster** corrections.



## Context sensitive error check with N-gram models and Add-One smoothing

### Building N-gram Language Models
The function `build_ngram_counts` constructs frequency distributions for n-grams (sequences of `n` words).

N-grams are useful for context-aware spelling correction, as they allow us to consider not just individual words but also their surrounding context.


In [270]:
def build_ngram_counts(token_list, n=2):
    """
    Build n-gram counts for a given n.
    Returns a Counter mapping from an n-tuple to a frequency count.
    """
    ngrams = []
    for i in range(len(token_list) - n + 1):
        ngram = tuple(token_list[i:i+n])
        ngrams.append(ngram)
    return Counter(ngrams)

In [271]:
import sys
from collections import Counter


bigram_counts = Counter()   # (w1, w2) -> frequency
trigram_counts = Counter()  # (w1, w2, w3) -> frequency
fourgram_counts = Counter() # (w1, w2, w3, w4) -> frequency
fivegram_counts = Counter() # (w1, w2, w3, w4, w5) -> frequency

def load_bigrams(file_path):
    """
    Reads a file of bigrams in the format:
      <freq> <word1> <word2>
    Updates bigram_counts and WORDS with new frequencies.
    """
    with open(file_path, 'r', encoding='latin-1') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 3:
                continue
            freq_str, w1, w2 = parts
            try:
                freq = int(freq_str)
            except ValueError:
                continue
            
            bigram_counts[(w1, w2)] += freq
            
            WORDS[w1] += freq
            WORDS[w2] += freq

def load_trigrams(file_path):
    """
    Reads a file of trigrams in the format:
      <freq> <word1> <word2> <word3>
    Updates trigram_counts and WORDS with new frequencies.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 4:
                continue
            
            freq_str, w1, w2, w3 = parts
            try:
                freq = int(freq_str)
            except ValueError:
                continue
            
            trigram_counts[(w1, w2, w3)] += freq
            
            WORDS[w1] += freq
            WORDS[w2] += freq
            WORDS[w3] += freq

def load_fourgrams(file_path):
    """
    Reads a file of four-grams in the format:
      <freq> <w1> <w2> <w3> <w4>
    Updates fourgram_counts and WORDS with new frequencies.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            
            freq_str, w1, w2, w3, w4 = parts
            try:
                freq = int(freq_str)
            except ValueError:
                continue
            
            fourgram_counts[(w1, w2, w3, w4)] += freq
            
            WORDS[w1] += freq
            WORDS[w2] += freq
            WORDS[w3] += freq
            WORDS[w4] += freq


def load_fivegrams(file_path):
    """
    Reads a file of 5-grams in the format:
      <freq> <w1> <w2> <w3> <w4> <w5>
    Updates fivegram_counts and WORDS with new frequencies.
    """
    with open(file_path, 'r', encoding='latin-1') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 6:
                continue
            freq_str, w1, w2, w3, w4, w5 = parts
            try:
                freq = int(freq_str)
            except ValueError:
                continue
            
            fivegram_counts[(w1, w2, w3, w4, w5)] += freq
            
            WORDS[w1] += freq
            WORDS[w2] += freq
            WORDS[w3] += freq
            WORDS[w4] += freq
            WORDS[w5] += freq

Here we will precomupe prefix counts to get them in `O(1)` instead of `O(n)` every time we compute probabilities.

In [272]:
from tqdm import tqdm

def build_prefix_counts(ngram_counts, order):
    """
    For n-grams of length 'order', build a dictionary that maps
    the (order-1)-tuple prefix to the sum of counts of all n-grams
    sharing that prefix.

    Uses tqdm to display a progress bar.
    """
    prefix_sum = defaultdict(int)

    for ngram, count in tqdm(ngram_counts.items(), 
                             desc=f"Building prefix counts for {order}-grams", 
                             total=len(ngram_counts)):
        prefix = ngram[:-1]
        prefix_sum[prefix] += count

    return prefix_sum

### Computing and Loading N-gram Frequencies
- The dataset is processed to extract unigrams, bigrams, trigrams, four-grams, and five-grams.
- Pretrained frequency distributions from external text files are loaded.
- Prefix counts are constructed for each type of n-gram.

These computations enable a more robust probability-based spelling correction system.


In [273]:
all_tokens = words(big_text)
bigram_counts = build_ngram_counts(all_tokens, n=2)
trigram_counts = build_ngram_counts(all_tokens, n=3)
fourgram_counts = build_ngram_counts(all_tokens, n=4)
fivegram_counts = build_ngram_counts(all_tokens, n=5)

load_bigrams("data/bigrams.txt")
load_bigrams("data/coca_ngrams_x2w.txt")
load_trigrams("data/coca_ngrams_x3w.txt")
load_fourgrams("data/coca_ngrams_x4w.txt")
load_fivegrams("data/fivegrams.txt")
load_fivegrams("data/coca_ngrams_x5w.txt")

unigram_counts = WORDS

prefix_count_2 = build_prefix_counts(bigram_counts, order=2)
prefix_count_3 = build_prefix_counts(trigram_counts, order=3)
prefix_count_4 = build_prefix_counts(fourgram_counts, order=4)
prefix_count_5 = build_prefix_counts(fivegram_counts, order=5)

N = sum(WORDS.values())
V = len(WORDS)

Building prefix counts for 5-grams: 100%|██████████| 2128531/2128531 [00:00<00:00, 3554618.86it/s]


### Calculating probabilities of N-grams with Add-One smoothing:
- Unigram: $P(w) = \frac{\text{count}(w) + 1}{\sum \text{count}(w) + V}$
- Bigram: $P(w_2 | w_1) = \frac{\text{count}(w_1, w_2) + 1}{\text{count}(w_1) + V}$
- Trigram: $P(w_3 | w_1, w_2) = \frac{\text{count}(w_1, w_2, w_3) + 1}{\text{count}(w_1, w_2) + V}$
- Fourgram: $P(w_4 | w_1, w_2, w_3) = \frac{\text{count}(w_1, w_2, w_3, w_4) + 1}{\text{count}(w_1, w_2, w_3) + V}$
- Fivegram: $P(w_5 | w_1, w_2, w_3, w_4) = \frac{\text{count}(w_1, w_2, w_3, w_4, w_5) + 1}{\text{count}(w_1, w_2, w_3, w_4) + V}$

In [274]:
def unigram_prob_add_one(w):
    """
    Returns the Laplace-smoothed probability of a word w.
    """
    numerator   = unigram_counts[(w,)] + 1
    denominator = sum(unigram_counts.values()) + V
    return numerator / denominator

In [275]:
def bigram_prob_add_one(w1, w2):
    """
    Returns the Laplace-smoothed probability for bigram (w1, w2).
    """
    bigram = (w1, w2)
    prefix = (w1,)  

    numerator   = bigram_counts[bigram] + 1
    denominator = prefix_count_2[prefix] + V 
    return numerator / denominator

In [276]:
def trigram_prob_add_one(w1, w2, w3):
    """
    Returns the Laplace-smoothed probability for trigram (w1, w2, w3).
    """
    trigram = (w1, w2, w3)
    prefix = (w1, w2)

    numerator   = trigram_counts[trigram] + 1
    denominator = prefix_count_3[prefix] + V
    return numerator / denominator

In [277]:
def fourgram_prob_add_one(w1, w2, w3, w4):
    """
    Returns the Laplace-smoothed probability for fourgram (w1, w2, w3, w4).
    """
    fourgram = (w1, w2, w3, w4)
    prefix = (w1, w2, w3)

    numerator   = fourgram_counts[fourgram] + 1
    denominator = prefix_count_4[prefix] + V
    return numerator / denominator

In [278]:
def fivegram_prob_add_one(w1, w2, w3, w4, w5):
    """
    Returns the Laplace-smoothed probability for fivegram (w1, w2, w3, w4, w5).
    """
    fivegram = (w1, w2, w3, w4, w5)
    prefix = (w1, w2, w3, w4)

    numerator   = fivegram_counts[fivegram] + 1
    denominator = prefix_count_5[prefix] + V
    return numerator / denominator

### Generalized N-gram Probability Function
The `ngram_prob_add_one` function dynamically computes probabilities for up to 5-grams.

Depending on the length of the context (previous words), it applies the appropriate probability function:
- If no context: Unigram probability.
- If one previous word: Bigram probability.
- If two previous words: Trigram probability.
- If three previous words: Four-gram probability.
- If four previous words: Five-gram probability.

In [279]:
import math

def ngram_prob_add_one(context, word):
    """
    Given a tuple 'context' (of length n-1, up to 4) and a 'word', 
    return Laplace-smoothed P(word | context).
    
    context: tuple of length 0..4 for up to 5-gram.
    word: the next word string.
    """
    n = len(context) + 1 
    
    if n == 1:
        return unigram_prob_add_one(word)
    
    elif n == 2:
        w1 = context[0]
        return bigram_prob_add_one(w1, word)
    
    elif n == 3:
        w1, w2 = context
        return trigram_prob_add_one(w1, w2, word)
    
    elif n == 4:
        w1, w2, w3 = context
        return fourgram_prob_add_one(w1, w2, w3, word)
    
    elif n == 5:
        w1, w2, w3, w4 = context
        return fivegram_prob_add_one(w1, w2, w3, w4, word)
    
    else:
        return 1.0 / V  


### Beam Search for Spelling Correction
A **beam search algorithm** is implemented to find the most probable sequence of words. This method:
- Keeps track of `beam_size` best possible sequences at each step.
- Generates alternative corrections only if a word is out-of-vocabulary (OOV).
- Uses **n-gram probabilities** to select the most likely correction sequence.
- Returns the most probable corrected sentence.

This approach improves accuracy by leveraging contextual information over multiple words.


In [280]:
def beam_search_correct(token_list, beam_size=3):
    """
    Beam search that only generates alternative candidate corrections
    if the token is not in the vocabulary (OOV).
    Otherwise, if it's in the vocabulary, we assume it's correct
    and keep only that token as the sole candidate.
    """
    candidate_lists = []
    
    for token in token_list:
        if token in WORDS: 
            cands = [token]
        else:
            cands = candidates(token)  
            if not cands:
                cands = [token]
        
        candidate_lists.append(list(cands))
    
    beam = [(0.0, [])]
    
    for i, cands in enumerate(candidate_lists):
        new_beam = []
        for log_prob_so_far, seq in beam:
            context = tuple(seq[-4:]) 
            for cand in cands:
                p = ngram_prob_add_one(context, cand)
                new_log_prob = math.log(p + 1e-15) + log_prob_so_far
                new_seq = seq + [cand]
                new_beam.append((new_log_prob, new_seq))
        
        new_beam.sort(key=lambda x: x[0], reverse=True)
        beam = new_beam[:beam_size]
    
    best_seq = beam[0][1]
    return best_seq


### Evaluating the Beam Search Spell Checker
The function `evaluate_solution(slice_size)` tests the beam search-based spell checker on a dataset.

The evaluation process:
- Reads a test dataset containing correct (`text`) and incorrect (`augmented_text`) sentences.
- Applies `beam_search_correct` to each sentence.
- Compares the output with the reference using **Word Error Rate (WER)**.
- Computes the overall accuracy of the model.

The final result indicates how well the beam search model performs in spelling correction.


In [ ]:
import pandas as pd
from jiwer import wer  

def evaluate_solution(slice_size):

    evaluation_data = pd.read_csv('data/test.csv')
    data = evaluation_data.head(slice_size)
    scores = []
    for idx, row in tqdm(data.iterrows(), desc= "Beam search evaluation", total=len(data)):
        ref = row["text"].lower()
        hyp = ' '.join(beam_search_correct(row['augmented_text'].lower().split(), beam_size=20))
        score = wer(ref, hyp) 
        acc = 1 - score
        scores.append(acc)


    overall_acc = sum(scores)/len(scores)
    print("Average Accuracy over dataset:", overall_acc)
    return overall_acc

evaluate_solution(slice_size=SLICE_SIZE)

Beam search evaluation: 100%|██████████| 10000/10000 [09:37<00:00, 17.32it/s]

Average Accuracy over dataset: 0.7324760128760129


0.7324760128760129

### Conclusion:
Our approach achieved a **3% improvement in accuracy**, which is a positive step forward. While this is a good result, we aim to further enhance performance.

Additionally, the new solution is **twice as fast** as the baseline, processing an average of **~17 rows per second**.

### Next Steps:
To further improve accuracy, we will:
- Introduce **interpolation** to compute probabilities more effectively.
- Implement **Kneser-Ney smoothing** to refine our probability estimates and enhance word prediction accuracy.


## N-gram models with Interpolation and Kneser-Ney smoothing

### Kneser-Ney Smoothing for Bigrams
The notebook implements **Kneser-Ney smoothing**, an advanced probability estimation technique for n-grams. It improves traditional models by considering word context in a more meaningful way.

- **Discounting (D):** Adjusts frequency counts to improve probability estimates.
- **P_continuation:** Measures how often a word appears in different contexts.
- **Alpha function:** Balances between higher-order and lower-order probabilities.

The `p_kn_bigram(w1, w2)` function applies this smoothing technique for bigrams.


In [282]:
from collections import defaultdict


D = 0.85


distinct_left_contexts = defaultdict(set)
for (w1, w2), count in bigram_counts.items():
    if count > 0:
        distinct_left_contexts[w2].add(w1)

continuation_count = {w: len(distinct_left_contexts[w]) for w in distinct_left_contexts}

prefix_count_2 = {}
prefix_bigrams_nonzero = {}

for (w1, w2), count in bigram_counts.items():
    prefix_count_2[w1] = prefix_count_2.get(w1, 0) + count
    if count > 0:
        prefix_bigrams_nonzero[w1] = prefix_bigrams_nonzero.get(w1, 0) + 1

num_bigram_types = sum(1 for (w1, w2), c in bigram_counts.items() if c>0)



def alpha_bigram(w1):
    """
    Normalization factor for bigram prefix w1.
    """
    # number of bigrams with prefix w1 that have freq>0
    unique_continuations = prefix_bigrams_nonzero.get(w1, 0)
    total_count_prefix = prefix_count_2.get(w1, 0)
    
    if total_count_prefix == 0:
        return 0.0
    
    return (D * unique_continuations) / total_count_prefix

def p_continuation(w):
    """
    Kneser–Ney continuation probability for w.
    p_continuation(w) = distinct_left_contexts[w] / num_bigram_types
    """
    c = continuation_count.get(w, 0)
    return c / num_bigram_types if num_bigram_types > 0 else 0.0

def p_kn_bigram(w1, w2):
    """
    Basic bigram Kneser–Ney:
    P(w2 | w1) = [max{c(w1,w2)-D,0} / sum_{u} c(w1,u)] + alpha(w1)*p_continuation(w2)
    """
    count_bigram = bigram_counts.get((w1, w2), 0)
    prefix_total = prefix_count_2.get(w1, 0)
    
    # Higher-order part:
    numerator = max(count_bigram - D, 0)
    if prefix_total > 0:
        higher_order_term = numerator / prefix_total
    else:
        higher_order_term = 0.0
    
    # interpolation weight alpha
    a = alpha_bigram(w1)
    
    cont = p_continuation(w2)
    
    return higher_order_term + a*cont

### Kneser-Ney Smoothing for Trigrams
The `p_kn_trigram(w1, w2, w3)` function extends the smoothing technique to trigrams:


$P(w_3 | w_1, w_2) = \frac{\max(c(w_1, w_2, w_3) - D, 0)}{\sum c(w_1, w_2, u)} + \alpha(w_1, w_2) P_{KN}(w_2, w_3)$


- Uses trigram counts but falls back to bigrams when needed.
- The **alpha function** ensures smooth transition between different n-gram orders.


In [283]:
prefix_count_3 = {}
trigram_prefix_nonzero = {}

for (w1, w2, w3), c in trigram_counts.items():
    prefix = (w1, w2)
    prefix_count_3[prefix] = prefix_count_3.get(prefix, 0) + c
    if c > 0:
        trigram_prefix_nonzero[prefix] = trigram_prefix_nonzero.get(prefix, 0) + 1

def alpha_trigram(w1, w2, D=0.75):
    prefix = (w1, w2)
    # number of distinct w3 s.t. c(w1, w2, w3)>0
    unique_continuations = trigram_prefix_nonzero.get(prefix, 0)
    total_count_prefix = prefix_count_3.get(prefix, 0)
    
    if total_count_prefix == 0:
        return 0.0
    
    return (D * unique_continuations) / total_count_prefix

def p_kn_trigram(w1, w2, w3):
    """
    Kneser–Ney trigram:
    P(w3 | w1, w2) = [max{ c(w1,w2,w3)-D, 0 } / sum_u c(w1,w2,u)]
                   + alpha(w1,w2) * p_kn_bigram(w2, w3)
    """
    trigram = (w1, w2, w3)
    c_tri = trigram_counts.get(trigram, 0)
    prefix = (w1, w2)
    total_prefix = prefix_count_3.get(prefix, 0)
    
    numerator = max(c_tri - D, 0)
    if total_prefix > 0:
        higher_order_term = numerator / total_prefix
    else:
        higher_order_term = 0.0
    
    # backoff to the KN bigram
    a = alpha_trigram(w1, w2, D=D)
    lower_order = p_kn_bigram(w2, w3)
    
    return higher_order_term + a*lower_order


## Interpolated Trigram Probability Model
The `interpolated_trigram` function combines different probability models:


$P(w_3 | w_1, w_2) = \lambda_3 P_{KN}(w_3 | w_1, w_2) + \lambda_2 P_{KN}(w_3 | w_2) + \lambda_1 P_{cont}(w_3)$

where:
- $ \lambda_3, \lambda_2, \lambda_1 $ control the weight of different models.

This interpolation allows for a robust n-gram language model.


In [284]:
def interpolated_trigram(w1, w2, w3, lambda1=0.2, lambda2=0.3, lambda3=0.5):
    """
    Returns the interpolated probability for w3 given up to 2-word context (w1,w2).
    If context is shorter, we handle that below.
    """
    p_tri = p_kn_trigram(w1, w2, w3)  # trigram
    p_bi  = p_kn_bigram(w2, w3)       # bigram (the immediate context)
    p_uni = p_continuation(w3)         # unigram
    
    return lambda3 * p_tri + lambda2 * p_bi + lambda1 * p_uni


### Beam Search with Trigram Context
This updated `beam_search_correct` function enhances spelling correction by:
- Using **trigram context** to improve word prediction.
- Applying **interpolated trigram probabilities** to balance between different n-gram orders.
- Implementing **beam search** to explore the best correction paths.

This technique ensures higher accuracy in real-world spelling correction tasks.


In [285]:
import math

def get_trigram_context(seq):
    """
    Returns exactly 2 tokens of context for trigram use,
    padding with <s> if seq has fewer than 2 tokens.
    """
    needed = 2 - len(seq)
    if needed > 0:
        pad = ["<s>"] * needed
        context = pad + seq
    else:
        context = seq
    
    return context[-2:]


def beam_search_correct(token_list, beam_size=3, lambda1=0.2, lambda2=0.3, lambda3=0.5):
    candidate_lists = []
    for token in token_list:
        if token in WORDS:
            cands = [token]
        else:
            cands = candidates(token)
            if not cands:
                cands = [token]
        candidate_lists.append(cands)

    beam = [(0.0, [])]

    for cands in candidate_lists:
        new_beam = []
        for (log_prob_so_far, seq_so_far) in beam:
            context = get_trigram_context(seq_so_far) 
            w1, w2 = context
            for cand in cands:
                p = interpolated_trigram(w1, w2, cand, 
                                         lambda1=lambda1, 
                                         lambda2=lambda2, 
                                         lambda3=lambda3)
                new_log_prob = log_prob_so_far + math.log(p + 1e-15)
                new_seq = seq_so_far + [cand]
                new_beam.append((new_log_prob, new_seq))
        new_beam.sort(key=lambda x: x[0], reverse=True)
        beam = new_beam[:beam_size]

    return beam[0][1]



### Evaluating the Final Model
The `evaluate_solution(slice_size=SLICE_SIZE)` function tests the final spell-checking model.

- Runs beam search with trigram probabilities on a test dataset.
- Computes **Word Error Rate (WER)** to measure performance.
- Reports the overall accuracy of the model.

The results indicate how well the new approach improves spelling correction.


In [286]:
evaluate_solution(slice_size=SLICE_SIZE)

Beam search evaluation: 100%|██████████| 10000/10000 [09:21<00:00, 17.82it/s]

Average Accuracy over dataset: 0.7938186677211677


0.7938186677211677

### Conclusion:
Achieving **79% accuracy** marks a significant improvement over the baseline. Additionally, this solution is **twice as fast**, further enhancing its practicality and efficiency.

# Deep Analysis of the Results  

## **1. Baseline: Classical Norvig Approach**
- **Accuracy:** 70%  
- **Speed:** Slow  

The classical Norvig spell-checker relies on:
1. **Word frequency models** to determine the most likely correction.
2. **Simple edit-distance-based candidate generation** (inserting, deleting, replacing, or transposing letters).
3. **Probability estimation** using word counts from a corpus.

### **Why is it limited?**
- **Context-Free:** It treats each word independently, ignoring the sentence context.
- **Limited Correction Scope:** The edit distance approach can sometimes miss phonetically similar words or words that require multiple edits.
- **Computationally Expensive:** Checking all possible edits and computing probabilities is slow for large datasets.

---

## **2. N-grams (up to 5) + Add-One Smoothing + Beam Search**
- **Accuracy:** 73%  
- **Speed:** 2× faster than baseline  

### **What Changed?**
1. **Incorporating N-grams (up to 5-grams)**  
   - Instead of treating each word in isolation, this model considers its neighboring words.
   - Higher-order n-grams (up to 5) allow the model to make context-aware corrections.
   
2. **Add-One (Laplace) Smoothing**  
   - Ensures that unseen word sequences get a small nonzero probability.
   - Helps generalize better but can introduce bias, as it doesn’t distinguish between frequent and rare sequences effectively.

3. **Beam Search**  
   - Instead of considering only the highest-probability word at each step, it keeps multiple candidate sequences and selects the best correction.
   - Reduces errors in longer sentences by maintaining contextual consistency.

### **Why is it better than the baseline?**
**Context Awareness**: The use of n-grams allows the model to understand which words are likely to appear together.  
**Higher Accuracy (74%)**: This method captures more word relationships, improving correction quality.  
**Faster (2× the baseline speed)**: Beam search efficiently prunes unpromising candidates, reducing unnecessary computations.

###  **Why is it still limited?**
**High-order N-grams Are Data-Hungry**: Using 4-grams or 5-grams requires a massive dataset to provide reliable probabilities.  
**Add-One Smoothing Is Too Simple**: It doesn’t effectively model word distributions and gives equal weight to rare and frequent words.

---

## **3. N-grams (Trigram) + Kneser-Ney Smoothing + Interpolation + Beam Search**
- **Accuracy:** 79%  
- **Speed:** 2× faster than baseline  

### **What Changed?**
1. **Using Trigrams Instead of Higher N-grams**  
   - While the previous model used up to 5-grams, this approach restricts itself to **trigrams**.
   - **Why?** Trigrams provide a balance between **context-awareness** and **data efficiency**. Using higher-order n-grams (4 or 5) introduces sparsity issues, requiring more data for reliable estimates.

2. **Kneser-Ney Smoothing**  
   - Instead of just adjusting counts like Add-One smoothing, **Kneser-Ney** considers how often a word appears in different contexts.  
   - Example:  
     - "New York" is common, so "York" should have a high probability when preceded by "New" even if "York" itself isn't frequent.
     - This makes it **better at handling rare words and unseen n-grams**.

3. **Interpolation**  
   - Instead of relying purely on trigram probabilities, the model **blends unigram, bigram, and trigram probabilities**.
   - Helps when the trigram is missing, falling back on bigram or unigram data.

4. **Beam Search for Sequence Correction**  
   - Continues optimizing for best word sequence rather than choosing words in isolation.

### **Why is it better than the previous approach?**
**Context-Aware Yet Efficient**: Trigrams provide enough context without suffering from data sparsity.  
**Better Generalization**: Kneser-Ney smoothing ensures rare and unseen word combinations are handled more effectively.  
**Highest Accuracy (80%)**: The combination of interpolation and Kneser-Ney provides the best probability estimates.  
**Still Fast (2× the baseline speed)**: Beam search optimizations keep speed high while improving accuracy.


## **Final Comparison Table:**
| **Model** | **Accuracy** | **Speed** | **Pros** | **Cons** |
|-----------|-------------|-----------|----------|----------|
| **Baseline (Norvig)** | 70% | Slow | Simple, frequency-based | No context awareness, slow |
| **N-grams (5) + Add-One + Beam Search** | 74% | 2× faster | Context-aware, faster, better corrections | Requires large corpus, Add-One smoothing is naive |
| **N-grams (3) + Kneser-Ney + Interpolation + Beam Search** | **80%** | **2× faster** | Best accuracy, efficient, handles unseen words well | Still dependent on training corpus quality |
